# Lesson 2: Using a String Template


#### Setup
Set the MakerSuite API key with the provided helper function.

In [1]:
import os
from utils import get_api_key
import google.generativeai as palm
from google.api_core import client_options as client_options_lib

palm.configure(
    api_key=get_api_key(),
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)

#### Pick the model that generates text

In [2]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model_bison = models[0]
model_bison

Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)

#### Helper function to call the PaLM API

In [3]:
from google.api_core import retry
@retry.Retry()
def generate_text(prompt, 
                  model=model_bison, 
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)

#### Prompt template

1. priming: getting the LLM ready for the type of task you'll ask it to do.
2. question: the specific task.
3. decorator: how to provide or format the output.

In [4]:
prompt_template = """
{priming}

{question}

{decorator}

Your solution:
"""

In [5]:
priming_text = "You are an expert at writing clear, concise, Python code."

In [6]:
question = "create a doubly linked list"

#### Observe how the decorator affects the output
- In other non-coding prompt engineering tasks, it's common to use "chain-of-thought prompting" by asking the model to work through the task "step by step".
- For certain tasks like generating code, you may want to experiment with other wording that would make sense if you were asking a developer the same question.

In the code cell below, try out option 1 first, then try out option 2.

In [7]:
# option 1
# decorator = "Work through it step by step, and show your work. One step per line."

# option 2
decorator = "Insert comments for each line of code."

In [8]:
prompt = prompt_template.format(priming=priming_text,
                                question=question,
                                decorator=decorator)

#### review the prompt

In [9]:
print(prompt)


You are an expert at writing clear, concise, Python code.

create a doubly linked list

Insert comments for each line of code.

Your solution:



#### Call the API to get the completion

In [10]:
completion = generate_text(prompt)
print(completion.result)

```python
class Node:

    """Node in a doubly linked list."""

    def __init__(self, data):
        """Initialize a node with the given data."""
        self.data = data
        self.next = None
        self.prev = None


class DoublyLinkedList:

    """Doubly linked list."""

    def __init__(self):
        """Initialize an empty doubly linked list."""
        self.head = None
        self.tail = None
        self.size = 0

    def __len__(self):
        """Return the number of nodes in the list."""
        return self.size

    def is_empty(self):
        """Return True if the list is empty."""
        return self.size == 0

    def add_first(self, data):
        """Add a new node with the given data to the front of the list."""
        new_node = Node(data)
        if self.is_empty():
            self.head = new_node
            self.tail = new_node
        else:
            new_node.next = self.head
            self.head.prev = new_node
            self.head = new_node
        se

#### Try another question

In [11]:
question = """create a very large list of random numbers in python, 
and then write code to sort that list"""

In [12]:
prompt = prompt_template.format(priming=priming_text,
                                question=question,
                                decorator=decorator)

In [13]:
print(prompt)


You are an expert at writing clear, concise, Python code.

create a very large list of random numbers in python, 
and then write code to sort that list

Insert comments for each line of code.

Your solution:



In [14]:
completion = generate_text(prompt)
print(completion.result)

```python
# Create a list of 100,000 random numbers between 0 and 100
random_numbers = [random.randint(0, 100) for _ in range(100000)]

# Sort the list of random numbers
sorted_random_numbers = sorted(random_numbers)

# Print the first 10 elements of the sorted list
print(sorted_random_numbers[:10])

```

**Explanation:**

* The first line of code creates a list of 100,000 random numbers between 0 and 100. The `random.randint()` function is used to generate a random integer between two specified values.
* The second line of code sorts the list of random numbers using the `sorted()` function. The `sorted()` function takes a list as its argument and returns a new list that is sorted in ascending order.
* The third line of code prints the first 10 elements of the sorted list. The `print()` function is used to print the contents of a list to the console.

This code is clear, concise, and efficient. It uses the built-in `random` and `sorted` functions to create and sort a list of random num

#### Try out the generated code
- Debug it as needed.  For instance, you may need to import `random` in order to use the `random.randint()` function.

In [16]:
# copy-paste some of the generated code that generates random numbers
import random
random_numbers = [random.randint(0, 100) for _ in range(100000)]
print(random_numbers)

[95, 38, 68, 23, 98, 84, 31, 21, 49, 10, 97, 61, 14, 91, 20, 73, 6, 75, 41, 49, 18, 8, 73, 35, 58, 47, 71, 14, 64, 8, 30, 8, 87, 61, 99, 25, 28, 73, 12, 62, 94, 50, 97, 76, 12, 17, 77, 91, 37, 54, 35, 87, 0, 66, 41, 48, 74, 100, 12, 28, 50, 42, 24, 9, 54, 74, 7, 5, 31, 38, 17, 99, 11, 45, 64, 26, 64, 68, 31, 32, 89, 86, 93, 37, 36, 13, 73, 38, 11, 99, 28, 33, 88, 29, 37, 58, 4, 97, 44, 70, 86, 98, 9, 87, 64, 30, 21, 62, 92, 87, 94, 23, 17, 42, 47, 3, 44, 39, 87, 2, 7, 48, 35, 68, 94, 15, 79, 57, 40, 56, 41, 69, 53, 23, 13, 99, 77, 44, 69, 5, 78, 53, 24, 15, 48, 64, 1, 50, 10, 80, 98, 0, 24, 18, 48, 42, 53, 38, 85, 8, 4, 55, 22, 27, 51, 38, 9, 61, 67, 12, 28, 54, 36, 55, 57, 60, 62, 32, 87, 28, 46, 58, 31, 53, 75, 55, 58, 52, 85, 96, 11, 1, 14, 89, 43, 99, 73, 32, 10, 74, 0, 78, 97, 22, 55, 23, 94, 11, 59, 38, 35, 91, 38, 61, 26, 10, 52, 75, 41, 80, 56, 70, 46, 28, 100, 22, 34, 52, 16, 5, 24, 79, 90, 34, 31, 87, 42, 22, 59, 91, 77, 29, 78, 71, 15, 52, 4, 16, 12, 48, 6, 31, 5, 36, 63, 55